# Histogram of GOES-16, GOES-17, Empirical CDF, and AUC for Full disk samples in region of interest.

## Import libraries

In [1]:
#import urllib.request
import numpy as np
import scipy.stats as st
import itertools
import pandas as pd
# npPath = '/sharedData/scratch/all_npy3/'
# ncPath = '/sharedData/scratch/april_data/'
# acmPath = '/sharedData/scratch/all_npy3/'
# DATAPATH = '/sharedData/scratch/'
import requests
import re
import os
#import os.path as op
from os import path as op
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib agg
%matplotlib agg
import xarray as xr
import metpy
import cartopy.crs as ccrs
from pyresample import image, geometry
import seaborn as sns #ref
import netCDF4
sns.set(style="darkgrid")
from pathlib import Path
from subprocess import Popen

import matplotlib.dates as mdates
from datetime import datetime

/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import logging
logger = logging.getLogger()

## Define paths

In [3]:
storage = Path('../storage/')

## Functions for unfiltered cloud mask

In [4]:
def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [5]:
def createUnfilteredPlotArray(ncFile,npFile,npPath):#Filtered Histrogram for cloud clear sky mask
    Tmean= []
    times = []
    for ncf, npf in zip(ncFile, npFile):
        imageBox = np.load(op.join(npPath,npf))
        myFile = xr.open_dataset(op.join(ncPath,ncf))
        planck_fk1 = float(myFile['planck_fk1'].data)
        planck_fk2 = float(myFile['planck_fk2'].data) 
        planck_bc1 = float(myFile['planck_bc1'].data)                       
        planck_bc2 = float(myFile['planck_bc2'].data)     
        T = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
        tString = ncf[31:38]
        times.append(tString)
        Tmean.append(T)
    return times, Tmean

In [6]:
def listurls(prefix,html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html.text)
    urllist = [elt['href'] for elt in soup.find_all(href=re.compile(prefix))]
    return urllist

In [7]:
def create_nc_Numpy(ncFile, pathOut):
    myFile = xr.open_dataset(ncFile,engine="netcdf4")
    dat = myFile.metpy.parse_cf('Rad')#myFile['Rad']
    geos = dat.metpy.cartopy_crs

    cartopy_extent_goes = geos.x_limits + geos.y_limits
    pyresample_extent_goes = (cartopy_extent_goes[0],
                                cartopy_extent_goes[2],
                                cartopy_extent_goes[1],
                                cartopy_extent_goes[3])
    goes_params = geos.proj4_params
    rad = dat.data
    
    def normIm(im,gamma=1.0,reverse=False):
        nim = ((im-np.nanmin(im))*(np.nanmax(im)-np.nanmin(im))**(-1))
        if reverse:#want clouds to be white
            nim = (1.0-nim**(gamma))
        return nim
    
    def goes_2_roi(geos_crs, 
               target_extent,
               target_rows,#actual length or base
               target_cols,#actual width or height
               cartopy_target_proj,
               data_key='Rad',
               radius_of_influence=50000):
        """Function that goes from loaded GOES data to data resampled in a projection for an extent"""
        cartopy_source_extent = geos_crs.x_limits + geos_crs.y_limits
        pyresample_source_extent = (cartopy_source_extent[0],
                                    cartopy_source_extent[2],
                                    cartopy_source_extent[1],
                                    cartopy_source_extent[3])
        rad = dat.data
        source_area = geometry.AreaDefinition('GOES-1X', 'Full Disk','GOES-1X', 
                                              geos_crs.proj4_params,
                                              rad.shape[1], rad.shape[0],
                                              pyresample_source_extent)
        area_target_def = geometry.AreaDefinition('areaTest', 'Target Region', 'areaTest',
                                            cartopy_target_proj.proj4_params,
                                            target_rows, target_cols,
                                            target_extent)
        geos_con_nn = image.ImageContainerNearest(rad, 
                                                source_area, 
                                                radius_of_influence=radius_of_influence)

        # Here we are using pyresample for the remapping
        area_proj_con_nn = geos_con_nn.resample(area_target_def)
        return area_proj_con_nn.image_data
        
    def cartopy_pyresample_toggle_extent(input_extent):
        return np.array(input_extent)[np.array([0,2,1,3])]

    def transform_cartopy_extent(source_extent,source_proj, target_proj):
        target_extent = target_proj.transform_points(source_proj, 
                                                     np.array(source_extent[:2]),
                                                     np.array(source_extent[2:])).ravel()
        # target_extent in 3D, must be in 2D
        return cartopy_pyresample_toggle_extent(np.array(target_extent)[np.array([0,1,3,4])])
    pc = ccrs.PlateCarree()
    mc = ccrs.Mercator()

    # Convert extent from pc to mc (both cylindrical projections)
    extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]
    
    target_extent_mc_cartopy = transform_cartopy_extent(extent_pc, pc, mc)
    target_extent_mc_pyresample = cartopy_pyresample_toggle_extent(target_extent_mc_cartopy)
    
    roi_rads = goes_2_roi(geos,
               target_extent_mc_pyresample,
               401,1001,
               mc)
    ####
    full_filename = op.join(pathOut,ncFile[:-3])
    np.save(full_filename,roi_rads)
    myFile.close()
    return

In [8]:
def download(url,toPath, saveName):
    cmd = [ 'wget ' + url +' -P ' + toPath +' -O '+ saveName]#if re.search('C07',url)
    print(cmd)
    pid = Popen(cmd, shell=True)
    pid.communicate()
    return

# Execution

In [9]:
Sat = 16
band = 8
year = 2020
month = 1
day = 1
hour = 0

In [ ]:
search = itertools.product([Sat], [band], [year], [month], [day], [hour])
bins = np.linspace(195,255,255-195)
filelog = open('log_Histogram.txt','w')

#check for GOES-16 and GOES-17 and plot what is available
for SS, bb, yyyy, mm, dd, hr in search:
    SS, bb, yyyy, mm, dd, hr = \
        str(SS).zfill(2),\
        str(bb).zfill(2),\
        str(yyyy).zfill(4),\
        str(mm).zfill(2),\
        str(dd).zfill(2),\
        str(hr).zfill(2)
    templateURL = f'http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/'\
    + f'goes16_download.cgi?source=aws&satellite='\
    + f'noaa-goes{SS}&domain=F&product=ABI-L1b-Rad&date={yyyy}-{mm}-{dd}&hour={hr}'
    
    '''Get URLS for download'''
    req16 = requests.get(templateURL)
   
    SS = '17'
    req17 = requests.get(templateURL)
    
    if yyyy == '2018':
        product = '3'
    else:
        product = '6'
    prefix = f"OR_ABI-L1b-RadF-M{product}C" + bb
    prefix.format(**{'product':product})
    bandURLList16 = listurls(prefix,req16)#list of strings using beautiful soup
    bandURLList17 = listurls(prefix,req17)
    bandURLList16.sort()
    bandURLList17.sort()
    
    cnt = 0 #keep a counter to avoid crossing FD with similar file names and performance
    '''iterate over pairs of GOES NETCDF, download, and convert to numpy'''
    #print(list(enumerate(bandURLList16)))
    for FD16 in bandURLList16: #Last string chars are of this format: c20192220009464.nc   
        print(FD16, file=filelog)
        
        YYYY = FD16[-17:-13]
        DDD = FD16[-13:-10]
        HH = FD16[-10:-8]
        i = FD16.find('_s')
        MM = FD16[i+11: i+13]
        ss = FD16[i+13: i+15]
        #print(f'{YYYY}{DDD}{HH}{MM}')
        
        _FD17 = None
        for FD17 in bandURLList17[cnt:i]:
            if FD16[i:13] == FD17[i:13]:
                cnt +=1
                _FD17 = FD17
                download17 = requests.get(FD17)
                nc17 = open('nc17.nc','wb')
                nc17.write(download17.content)
                nc17.close()
                myFile_17 = xr.open_dataset('nc17.nc')
                npy17 = create_nc_Numpy('nc17.nc', '.')
                
                '''Load numpy files'''
                imageBox_17 = np.load('nc17.npy')
                planck_fk1_17 = float(myFile_17['planck_fk1'].data)
                planck_fk2_17 = float(myFile_17['planck_fk2'].data) 
                planck_bc1_17 = float(myFile_17['planck_bc1'].data)                       
                planck_bc2_17 = float(myFile_17['planck_bc2'].data)
                myFile_17.close()
                break
            else:
                _FD17 = None
                imageBox_17 = None
                planck_fk1_17 = None
                planck_fk2_17 = None
                planck_bc1_17 = None
                planck_bc2_17 = None
        download16 = requests.get(FD16)
        nc16 = open('nc16.nc','wb')
        nc16.write(download16.content)
        nc16.close()
        npy16 = create_nc_Numpy('nc16.nc', '.')

        '''Load numpy files'''
        imageBox_16 = np.load('nc16.npy')
        myFile_16 = xr.open_dataset('nc16.nc')
        planck_fk1_16 = float(myFile_16['planck_fk1'].data)
        planck_fk2_16 = float(myFile_16['planck_fk2'].data) 
        planck_bc1_16 = float(myFile_16['planck_bc1'].data)                       
        planck_bc2_16 = float(myFile_16['planck_bc2'].data)
        myFile_16.close()
 
        '''Create subplot figures and save'''
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(15,15))
        
        '''Histograms'''
        ax1.set_title(f'GOES-16 ABI Band {bb} (6.2µm) {mm}-{dd}-{YYYY} {HH}:{MM} UTC')
        ax1.set_ylabel('density (unitless)',fontsize = 16)
        ax1.set_xlabel('Temperature (K)', fontsize = 16)
        Tbox16 = Rad2BT(imageBox_16, planck_fk1_16, planck_fk2_16, planck_bc1_16, planck_bc2_16)
        ax1.hist(Tbox16.ravel(), bins = bins, density = True, label = 'Temp dist.')
        ax1.axvline(Tbox16.mean(), color='r',label = 'NOAA avg.')
        ax1.legend(loc='upper left')
        ax1.axis(ymin=0, ymax=0.25)# Check July 15, 2020, 197th day of the year       
        
        if None != _FD17:
            ax2.set_title(f'GOES-17 ABI Band {bb} (6.2µm) {mm}-{dd}-{YYYY} {HH}:{MM} UTC')
            ax2.set_ylabel('density (unitless)',fontsize = 16)
            ax2.set_xlabel('Temperature (K)', fontsize = 16)
            Tbox17 = Rad2BT(imageBox_17, planck_fk1_17, planck_fk2_17, planck_bc1_17, planck_bc2_17)
            ax2.hist(Tbox17.ravel(), bins = bins, density = True, label = 'Temp dist.')
            ax2.axvline(Tbox17.mean(), color='r',label = 'NOAA avg.')
            ax2.legend(loc='upper left')
            ax2.axis(ymin=0, ymax=0.25)# Check July 15, 2020, 197th day of the year
            
            rc = np.vstack([Tbox16.ravel(), Tbox17.ravel()])
            XY = rc[:,np.isfinite(rc).all(axis=0)]
            G16 = XY[0]
            G17 = XY[1]
            plabels = ['0%', '25%', '50%', '75%', '100%']
            
            f'''Empirical CDF {mm}-{dd}-{YYYY} {HH}:{MM} UTC'''
            xticks = np.arange(len(G17))
            xticks = [st.scoreatpercentile(xticks, p) for p in [0, 25, 50, 75, 100]]
            ax3.plot(sorted(G17), label='G17')
            ax3.plot(sorted(G16), label='G16')
            ax3.legend(loc = 'upper left')
            ax3.set_title(f'Empirical CDF for {dd}-{mm}-{yyyy}')
            ax3.set_xticks(xticks)
            ax3.set_xticklabels(plabels)
            ax3.set_xlabel('quantiles')
            ax3.set_ylabel('Temperature (K)')
            ax3.axis(ymin=195, ymax=255)
            
            '''Area Under Curve (AUC)'''
            xRange = round(255-195)# Check July 15, 2020, 197th day of the year
            _G17 = np.linspace(G17.min(), G17.max(), xRange)
            B = [st.scoreatpercentile(G16, st.percentileofscore(G17,g17s,
                kind='strict')) for g17s in _G17]

            _B = (B - min(B))/(max(B) - min(B))
            _G17n = (_G17 - min(_G17))/(max(_G17) - min(_G17))

            AUC = round(sum(_B)/xRange,5)#Area under curve is sum of x * dx
            
            ax4.plot(_G17n,_B, label=f"AUC: {AUC}")
            ax4.set_ylabel('G16')
            ax4.set_xlabel('G17')
            ax4.set_title('Transform of G17 to G16')
            ax4.set_aspect('equal')
            ax4.legend(loc = 'upper left')
        fig.suptitle('Ronald O. Adomako, City College of New York, NOAA-CESSRST', fontsize=12)
        fig.savefig(op.join(storage,f"GOES-R_panel_GOES16vsGOES17_FD_{YYYY}{DDD}_{HH}{MM}{ss}_Band{bb}.png"))
